In [63]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
from together import Together
import os
load_dotenv()

True

In [64]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model="Qwen/Qwen3-235B-A22B-fp8-tput",
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [65]:
client = Together(
          api_key="tgp_v1_2zvLSlfkdH-aL1HBA17dg6TdpYgmB5nqXFWyovUK6oo",
            
        )
model_name = os.getenv("MODEL_NAME")

In [66]:


client = Together(api_key="tgp_v1_2zvLSlfkdH-aL1HBA17dg6TdpYgmB5nqXFWyovUK6oo")  # no comma here

response = client.chat.completions.create(
    model="Qwen/Qwen3-235B-A22B-fp8-tput",
    messages=[{"role": "user", "content": "What is the Capital of Germany?"}]
)

print(response.choices[0].message.content)


<think>
Okay, so the user is asking, "What is the Capital of Germany?" Let me think about how to approach this.

First, I know that Germany is a country in Europe, and like most countries, it has a capital city. But wait, wasn't there some history with Germany having different capitals at different times? Like, before reunification, West Germany had Bonn as its capital, and East Germany had East Berlin. After they reunified in 1990, they had to decide on a single capital.

I remember that Berlin was the capital of the unified Germany before World War II. After the war, it was divided into East and West Berlin. When the Berlin Wall fell in 1989 and Germany reunified, they moved the capital back to Berlin. But I think some government functions remained in Bonn for a while. There was a process called the "Bonn-Berlin Act" in 1994 that outlined moving the capital functions back to Berlin. The process was completed in the late 90s or early 2000s.

So now, Berlin is definitely the capital. B

# Get LLM response

In [67]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, so the user is asking, "What's the capital of Italy?" Let me think. I know Italy is a country in Europe, shaped like a boot. The capital is the city where the government is based, right? I remember that some people might confuse Rome with other cities like Florence or Venice, which are also in Italy and are well-known for different reasons. Florence is known for art, Venice for its canals. But the capital... I think it's Rome. Wait, wasn't Rome the ancient city, the center of the Roman Empire? Yeah, that's right. But when did it become the capital? I think Italy became a unified country in the 19th century, and Rome was chosen as the capital later. Before that, maybe Turin or Florence? But the current capital is definitely Rome. Let me double-check. Quick search in my mind: yes, Rome has been the capital since 1870. The Vatican City is within Rome, but that's a separate country. So the answer should be Rome. But wait, sometimes people might think it's Florence because of 

# Prompt engineering

## Structred output

In [68]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':"What's the capital of Italy?"})
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, the user is asking for the capital of Italy. Let me think. I know that Italy is a country in Europe, and I'm pretty sure its capital is Rome. Wait, let me double-check that. Sometimes people confuse it with other cities like Florence or Venice, but no, the capital is definitely Rome. I remember that Rome is where the government is based, and it's also a major historical city with landmarks like the Colosseum. Yeah, I'm confident that's right. So the answer should be Rome.
</think>

[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [69]:
import json
import re

# Assume `response` is a string containing the full output
# Extract JSON array using regex
match = re.search(r"\[\s*{.*?}\s*\]", response, re.DOTALL)
if match:
    json_str = match.group(0)
    data = json.loads(json_str)
    print(data)
else:
    print("No valid JSON array found in the response.")


[{'country': 'Italy', 'capital': 'Rome'}]


## input structuring

In [70]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, the user is asking for the capitals of Italy, France, and Germany. Let me make sure I remember these correctly. Italy's capital is Rome, right? Yeah, Rome is the capital. For France, the capital is Paris. That's pretty straightforward. Now Germany, that one I should double-check. I think it's Berlin. Yes, Berlin became the capital after reunification. So all three are Rome, Paris, and Berlin. Let me structure the JSON as specified, with each country and their respective capital. Make sure the order matches the list the user provided. No other text, just the JSON array.
</think>

[
  {
    "country": "Italy",
    "capital": "Rome"
  },
  {
    "country": "France",
    "capital": "Paris"
  },
  {
    "country": "Germany",
    "capital": "Berlin"
  }
]


In [71]:
import json
import re

# Assume `response` is a string containing the full output
# Extract JSON array using regex
match = re.search(r"\[\s*{.*?}\s*\]", response, re.DOTALL)
if match:
    json_str = match.group(0)
    data = json.loads(json_str)
    print(data)
else:
    print("No valid JSON array found in the response.")


[{'country': 'Italy', 'capital': 'Rome'}, {'country': 'France', 'capital': 'Paris'}, {'country': 'Germany', 'capital': 'Berlin'}]


## Give the model time to think (Chain of thought)

> https://arxiv.org/pdf/2205.11916

In [72]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, let's see. The user wants me to calculate 1 plus 3. That's a simple addition problem. So, 1 plus 3 equals 4. The result should be a JSON object with the key "result" and the value 4. I need to make sure the JSON is correctly formatted without any extra text. Just the object. Alright, I'll structure it as {"result": 4}.
</think>

{
    "result": 4
}


In [73]:
259/2*8654+91072*33-12971

4113098.0

In [74]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, let's tackle this equation step by step. The problem is 259 divided by 2 multiplied by 8654, plus 91072 multiplied by 33, minus 12971. I need to calculate each part correctly following the order of operations.

First, division and multiplication have the same precedence and are evaluated left to right. So starting with 259/2. Let me calculate that. 259 divided by 2 is 129.5. Then I need to multiply that by 8654. Let me write that down: 129.5 * 8654. Hmm, that's a big number. Let me do the multiplication step by step. 129 * 8654 is... wait, maybe break it down. 100*8654 = 865,400, 20*8654=173,080, 9*8654=77,886. Adding those together: 865,400 + 173,080 = 1,038,480; plus 77,886 gives 1,116,366. Then the 0.5 part: 0.5 * 8654 = 4,327. So total for 129.5*8654 is 1,116,366 + 4,327 = 1,120,693. 

Next part is 91072 multiplied by 33. Let me calculate that. 90,000 * 33 = 2,970,000. 1,072 * 33: 1,000*33=33,000, 72*33=2,376. So 33,000 + 2,376 = 35,376. Adding to 2,970,000 gives 2,97

In [75]:
4113098.0 - 1431449.0

2681649.0

In [76]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [77]:
print(response)

<think>
Okay, let's tackle this equation step by step. The equation is 259 divided by 2 multiplied by 8654, plus 91072 multiplied by 33, minus 12971. I need to follow the order of operations, which is BEDMAS—Brackets, Exponents, Division and Multiplication from left to right, Addition and Subtraction from left to right.

First, looking at the equation: 259/2*8654 + 91072*33 - 12971. There are no brackets or exponents, so I start with division and multiplication. 

Starting with the first part: 259 divided by 2. Let me calculate that. 259 ÷ 2 equals 129.5. Then, I need to multiply that result by 8654. So 129.5 multiplied by 8654. Hmm, that's a big number. Let me do that step by step. Maybe break down 8654 into 8000 + 600 + 50 +4. So 129.5 * 8000 = 1,036,000. Then 129.5 * 600 = 77,700. 129.5 * 50 = 6,475. And 129.5 *4 = 518. Adding those together: 1,036,000 + 77,700 = 1,113,700. Then +6,475 = 1,120,175. Then +518 = 1,120,693. So the first part gives 1,120,693.

Next, the second part is 9

In [78]:
4113098.0 - 4113056

42.0

# RAG - Retrieval Augmented Generation

#### Asking about a subject that the LLM does not know anything about

In [79]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, so I need to figure out what's new in the iPhone 16. Let me start by recalling what I know about recent iPhone models. The latest one right now is the iPhone 15, which was released in September 2023. The iPhone 16 is expected to come out in September 2024, so there's no official information yet. But there are probably a lot of rumors and leaks about it. I should check those, but I need to be careful because not all of them might be accurate.

First, design changes. I remember that the iPhone 15 Pro had a titanium design and a new action button. Maybe the iPhone 16 will have similar changes across all models? Or maybe even more drastic design changes. Some rumors say that Apple might be moving towards a more curved design or maybe a different material. Also, the notch has been getting smaller each year; maybe it'll be gone or replaced with a pill-shaped cutout for the Dynamic Island, which was introduced in the iPhone 14 Pro.

Display technology could be another area. The 

#### Giving Context to the unknown subject

In [80]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [81]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, so I need to figure out what's new in the iPhone 16 based on the provided information. Let me start by reading through the text carefully. 

First, the user mentioned the iPhone 16 has several updates, making it Apple's most advanced smartphone. The base model has a 6.1-inch display, and the Plus is 6.7 inches. Both have thinner bezels and a more durable Ceramic Shield. The Pro and Pro Max have even larger screens at 6.3 and 6.9 inches with the thinnest bezels ever. 

Then there's the A18 chip for the base models and A18 Pro for the Pros. These chips offer better performance, enhanced neural engine, faster GPU for gaming, and machine learning improvements. 

The camera systems are upgraded. The base model has a dual-camera setup with a 48MP main sensor. The Pro models have a 48MP Ultra Wide and a 5x telephoto camera. There's also a new "Camera Control" button for more photography flexibility. 

Audio features include "Audio Mix," which uses machine learning to separate ba

#### Automatically extract context data from database

In [82]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [83]:
data = [iphone_16,samsung_s23]

In [84]:
user_prompt = """What's new in iphone 16?"""

In [85]:
embedding_client = Together(
        api_key=os.getenv("TOGETHER_API_KEY"), 
       
    )

In [86]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model="togethercomputer/m2-bert-80M-32k-retrieval")
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [87]:
user_prompt_embeddings = get_embedding(embedding_client,model_name,user_prompt)

In [88]:
user_prompt_embeddings

[[-0.015363121,
  -0.04879201,
  0.018760493,
  -0.017484726,
  0.01741185,
  -0.00039882582,
  0.08372597,
  -0.13339634,
  0.0036312703,
  0.008201296,
  -0.027981538,
  0.001641747,
  0.027784701,
  -0.12520237,
  0.07645388,
  0.08363384,
  -0.01610011,
  -0.005321186,
  0.0002663005,
  -0.031487104,
  0.015084278,
  0.07433317,
  -0.07803347,
  -0.024633622,
  0.01852701,
  -0.05747377,
  0.0071324613,
  0.07600312,
  -0.005203596,
  0.011805656,
  0.028511109,
  0.07097643,
  -0.05324066,
  -0.013709002,
  0.03852891,
  0.0074450993,
  -0.02363188,
  0.042420544,
  0.08773473,
  0.0016848466,
  0.008172077,
  0.038541634,
  0.00048264858,
  -0.022748617,
  0.029440533,
  -0.0031153683,
  0.028720878,
  0.029277809,
  0.058314167,
  0.084712885,
  -0.0045197913,
  0.024711618,
  -0.079630144,
  0.06669046,
  0.016101671,
  -0.02053872,
  0.087110594,
  0.08784563,
  0.034474064,
  0.003951827,
  -0.018765861,
  -0.07193985,
  0.05541608,
  0.01749174,
  0.031809054,
  -0.023758067

In [89]:
user_prompt_embeddings = user_prompt_embeddings[0]

In [90]:
data_embeddings = [get_embedding(embedding_client,model_name,x)[0] for x in data]

In [91]:
data_similaraty_scores = cosine_similarity([user_prompt_embeddings], data_embeddings)

In [92]:
data_similaraty_scores

array([[0.89700992, 0.92557888]])

In [93]:
closest_entry_index=data_similaraty_scores.argmax()
closest_entry_index

1

In [94]:
data[closest_entry_index]

"\nThe Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra\u200b\nSTUFF\n\nTECHRADAR\n.\n\nIn terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops\u200b\nTECHRADAR\n.\n\nThe S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also inclu

In [95]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [96]:
user_prompt_with_data

"\n\nThe Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra\u200b\nSTUFF\n\nTECHRADAR\n.\n\nIn terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops\u200b\nTECHRADAR\n.\n\nThe S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also inc

In [97]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

<think>
Okay, the user just asked, "What's new in iPhone 16?" after I provided information about the Samsung Galaxy S23 series. Let me start by recalling the details I have on the iPhone 16. Wait, but the iPhone 16 hasn't been released yet, right? The latest model is the iPhone 15 series, which came out in September 2023. So the iPhone 16 would be the next generation, expected in late 2024. Since it's not out yet, there's no official information from Apple. Therefore, any details about the iPhone 16 would be speculative, based on rumors and leaks.

I need to make sure I don't present any unverified information as fact. The user might be under the impression that the iPhone 16 is already available or that there's confirmed news about it. My response should clarify that the iPhone 16 hasn't been announced yet and that any information is speculative. 

Looking at the previous conversation, the user was interested in the Galaxy S23's features like the processor, design, camera, and battery